## Second Notebook Dealing with Intraday Demand

# Was having issues with the merge on the forecastdemand data - looks like there were duplciates in the original DF that was causing issues. 

In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
forecastdemand_vic = pd.read_csv("../data/Australia/a/forecastdemand_vic.csv")
forecastdemand_sa = pd.read_csv("../data/Australia/b/forecastdemand_sa.csv")
forecastdemand_qld = pd.read_csv("../data/Australia/c/forecastdemand_qld.csv")
temperature_qld = pd.read_csv("../data/Australia/d/temprature_qld.csv")  # There's a typo in 'temperature' in the file path
temperature_sa = pd.read_csv("../data/Australia/d/temprature_sa.csv")  # Same typo as above
temperature_vic = pd.read_csv("../data/Australia/d/temprature_vic.csv")  # Same typo as above
totaldemand_qld = pd.read_csv("../data/Australia/d/totaldemand_qld.csv")
totaldemand_sa = pd.read_csv("../data/Australia/d/totaldemand_sa.csv")
totaldemand_vic = pd.read_csv("../data/Australia/d/totaldemand_vic.csv")

In [38]:
dataframes = [forecastdemand_qld, forecastdemand_vic, totaldemand_sa, totaldemand_qld, temperature_vic, temperature_sa, temperature_qld, totaldemand_vic]

for df in dataframes:
    df['DATETIME'] = pd.to_datetime(df['DATETIME'])

for df in dataframes:
    print(df.head())

<ipython-input-38-1868340a0ddf>:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DATETIME'] = pd.to_datetime(df['DATETIME'])


   PREDISPATCHSEQNO REGIONID  PERIODID  FORECASTDEMAND          LASTCHANGED  \
0        2016123018     QLD1        71         6425.36  2016-12-30 12:31:25   
1        2016123018     QLD1        72         6193.34  2016-12-30 12:31:25   
2        2016123018     QLD1        73         6016.45  2016-12-30 12:31:25   
3        2016123018     QLD1        74         5859.23  2016-12-30 12:31:25   
4        2016123018     QLD1        75         5767.15  2016-12-30 12:31:25   

             DATETIME  YEAR  MONTH  DAY  HOUR  MINUTE  
0 2017-01-01 00:00:00  2017      1    1     0       0  
1 2017-01-01 00:30:00  2017      1    1     0      30  
2 2017-01-01 01:00:00  2017      1    1     1       0  
3 2017-01-01 01:30:00  2017      1    1     1      30  
4 2017-01-01 02:00:00  2017      1    1     2       0  
   PREDISPATCHSEQNO REGIONID  PERIODID  FORECASTDEMAND          LASTCHANGED  \
0        2016123018     VIC1        71         4160.74  2016-12-30 12:31:25   
1        2016123018     VIC1   

In [6]:
forecastdemand_qld['DATETIME'] = pd.to_datetime(forecastdemand_qld['DATETIME'], format="%d/%m/%Y %H:%M")
temperature_qld['DATETIME'] = pd.to_datetime(temperature_qld['DATETIME'], format="%d/%m/%Y %H:%M")
totaldemand_qld['DATETIME'] = pd.to_datetime(totaldemand_qld['DATETIME'], format="%Y-%m-%d %H:%M:%S")

merged_df_1 = pd.merge(forecastdemand_qld, temperature_qld, on='DATETIME', how='inner')
df_QLD = pd.merge(merged_df_1, totaldemand_qld, on='DATETIME', how='inner')

df_QLD['forecast_error'] = df_QLD['FORECASTDEMAND'] - df_QLD['TOTALDEMAND']
df_QLD['forecast_error_ratio'] = df_QLD['forecast_error'] / df_QLD['FORECASTDEMAND']

print(df_QLD.head(10))



   PREDISPATCHSEQNO REGIONID_x  PERIODID  FORECASTDEMAND          LASTCHANGED  \
0        2016123018       QLD1        71         6425.36  2016-12-30 12:31:25   
1        2016123019       QLD1        70         6425.42  2016-12-30 13:01:33   
2        2016123020       QLD1        69         6425.42  2016-12-30 13:31:24   
3        2016123021       QLD1        68         6427.40  2016-12-30 14:01:21   
4        2016123022       QLD1        67         6426.99  2016-12-30 14:31:22   
5        2016123023       QLD1        66         6426.63  2016-12-30 15:01:26   
6        2016123024       QLD1        65         6426.63  2016-12-30 15:31:28   
7        2016123025       QLD1        64         6425.73  2016-12-30 16:01:13   
8        2016123026       QLD1        63         6426.89  2016-12-30 16:31:13   
9        2016123027       QLD1        62         6426.80  2016-12-30 17:01:13   

    DATETIME                      LOCATION  TEMPERATURE  TOTALDEMAND  \
0 2017-01-01  Brisbane Archerfield A

In [7]:
from functools import reduce
dataframes_qld = [forecastdemand_qld, totaldemand_qld, temperature_qld]
df_qld_merged = reduce(lambda left, right: pd.merge(left, right, on='DATETIME', how='inner'), dataframes_qld)
print(df_qld_merged.head())



   PREDISPATCHSEQNO REGIONID_x  PERIODID  FORECASTDEMAND          LASTCHANGED  \
0        2016123018       QLD1        71         6425.36  2016-12-30 12:31:25   
1        2016123019       QLD1        70         6425.42  2016-12-30 13:01:33   
2        2016123020       QLD1        69         6425.42  2016-12-30 13:31:24   
3        2016123021       QLD1        68         6427.40  2016-12-30 14:01:21   
4        2016123022       QLD1        67         6426.99  2016-12-30 14:31:22   

    DATETIME  TOTALDEMAND REGIONID_y                      LOCATION  \
0 2017-01-01      6728.99       QLD1  Brisbane Archerfield Airport   
1 2017-01-01      6728.99       QLD1  Brisbane Archerfield Airport   
2 2017-01-01      6728.99       QLD1  Brisbane Archerfield Airport   
3 2017-01-01      6728.99       QLD1  Brisbane Archerfield Airport   
4 2017-01-01      6728.99       QLD1  Brisbane Archerfield Airport   

   TEMPERATURE  
0         26.5  
1         26.5  
2         26.5  
3         26.5  
4      

In [8]:
dataframes_qld = [forecastdemand_qld, totaldemand_qld, temperature_qld]

for df in dataframes_qld:
    df['YEAR'] = df['DATETIME'].dt.year
    df['MONTH'] = df['DATETIME'].dt.month
    df['DAY'] = df['DATETIME'].dt.day
    df['HOUR'] = df['DATETIME'].dt.hour
    df['MINUTE'] = df['DATETIME'].dt.minute

merge_keys = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']
df_qld_merged = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how='inner'), dataframes_qld)

print(df_qld_merged.head())



   PREDISPATCHSEQNO REGIONID_x  PERIODID  FORECASTDEMAND          LASTCHANGED  \
0        2016123018       QLD1        71         6425.36  2016-12-30 12:31:25   
1        2016123019       QLD1        70         6425.42  2016-12-30 13:01:33   
2        2016123020       QLD1        69         6425.42  2016-12-30 13:31:24   
3        2016123021       QLD1        68         6427.40  2016-12-30 14:01:21   
4        2016123022       QLD1        67         6426.99  2016-12-30 14:31:22   

  DATETIME_x  YEAR  MONTH  DAY  HOUR  MINUTE DATETIME_y  TOTALDEMAND  \
0 2017-01-01  2017      1    1     0       0 2017-01-01      6728.99   
1 2017-01-01  2017      1    1     0       0 2017-01-01      6728.99   
2 2017-01-01  2017      1    1     0       0 2017-01-01      6728.99   
3 2017-01-01  2017      1    1     0       0 2017-01-01      6728.99   
4 2017-01-01  2017      1    1     0       0 2017-01-01      6728.99   

  REGIONID_y                      LOCATION   DATETIME  TEMPERATURE  
0       QLD

In [9]:
print("Forecast Demand QLD 'DATETIME' Sample and Dtype:")
print(forecastdemand_qld['DATETIME'].head())
print(forecastdemand_qld['DATETIME'].dtype)

print("\nTemperature QLD 'DATETIME' Sample and Dtype:")
print(temperature_qld['DATETIME'].head())
print(temperature_qld['DATETIME'].dtype)

print("\nTotal Demand QLD 'DATETIME' Sample and Dtype:")
print(totaldemand_qld['DATETIME'].head())
print(totaldemand_qld['DATETIME'].dtype)


Forecast Demand QLD 'DATETIME' Sample and Dtype:
0   2017-01-01 00:00:00
1   2017-01-01 00:30:00
2   2017-01-01 01:00:00
3   2017-01-01 01:30:00
4   2017-01-01 02:00:00
Name: DATETIME, dtype: datetime64[ns]
datetime64[ns]

Temperature QLD 'DATETIME' Sample and Dtype:
0   2010-01-01 00:00:00
1   2010-01-01 00:30:00
2   2010-01-01 01:00:00
3   2010-01-01 01:30:00
4   2010-01-01 02:00:00
Name: DATETIME, dtype: datetime64[ns]
datetime64[ns]

Total Demand QLD 'DATETIME' Sample and Dtype:
0   2010-01-01 00:00:00
1   2010-01-01 00:30:00
2   2010-01-01 01:00:00
3   2010-01-01 01:30:00
4   2010-01-01 02:00:00
Name: DATETIME, dtype: datetime64[ns]
datetime64[ns]


In [10]:
unique_forecastdemand = forecastdemand_qld['DATETIME'].nunique()
unique_temperature = temperature_qld['DATETIME'].nunique()
unique_totaldemand = totaldemand_qld['DATETIME'].nunique()

print(f"Unique DATETIME in forecastdemand_qld: {unique_forecastdemand}")
print(f"Unique DATETIME in temperature_qld: {unique_temperature}")
print(f"Unique DATETIME in totaldemand_qld: {unique_totaldemand}")


print(forecastdemand_qld.shape)
print(totaldemand_qld.shape)
print(temperature_qld.shape)


Unique DATETIME in forecastdemand_qld: 73833
Unique DATETIME in temperature_qld: 208085
Unique DATETIME in totaldemand_qld: 196513
(4095592, 11)
(196513, 8)
(208085, 8)


In [11]:
datetime_counts_forecastdemand = forecastdemand_qld['DATETIME'].value_counts()
print(datetime_counts_forecastdemand.head(10))
print("\nDistribution of DATETIME occurrence counts:")
print(datetime_counts_forecastdemand.describe())


DATETIME
2020-04-07 04:00:00    79
2020-01-30 04:00:00    79
2019-10-19 04:00:00    79
2020-07-05 04:00:00    79
2020-11-16 04:00:00    79
2018-01-22 04:00:00    79
2019-06-01 04:00:00    79
2018-12-02 04:00:00    79
2019-10-18 04:00:00    79
2020-07-04 04:00:00    79
Name: count, dtype: int64

Distribution of DATETIME occurrence counts:
count    73833.000000
mean        55.471022
std         13.871803
min         23.000000
25%         43.000000
50%         55.000000
75%         67.000000
max         79.000000
Name: count, dtype: float64


In [12]:
specific_datetime_df = forecastdemand_qld[forecastdemand_qld['DATETIME'] == '2019-08-02 04:00:00']
print(specific_datetime_df)


         PREDISPATCHSEQNO REGIONID  PERIODID  FORECASTDEMAND  \
2509378        2019073118     QLD1        79         5199.24   
2509435        2019073119     QLD1        78         5196.16   
2509492        2019073120     QLD1        77         5192.53   
2509549        2019073121     QLD1        76         5192.46   
2509606        2019073122     QLD1        75         5192.41   
...                   ...      ...       ...             ...   
2513466        2019080144     QLD1         5         5129.04   
2513518        2019080145     QLD1         4         5121.99   
2513569        2019080146     QLD1         3         5127.00   
2513619        2019080147     QLD1         2         5124.00   
2513668        2019080148     QLD1         1         5107.55   

                 LASTCHANGED            DATETIME  YEAR  MONTH  DAY  HOUR  \
2509378  2019-07-31 12:31:29 2019-08-02 04:00:00  2019      8    2     4   
2509435  2019-07-31 13:01:34 2019-08-02 04:00:00  2019      8    2     4   
250

In [13]:
forecastdemand_qld.drop_duplicates(subset='DATETIME', inplace=True)
print(forecastdemand_qld.shape)


(73833, 11)


In [14]:
dataframes_qld = [forecastdemand_qld, totaldemand_qld, temperature_qld]

for df in dataframes_qld:
    df['YEAR'] = df['DATETIME'].dt.year
    df['MONTH'] = df['DATETIME'].dt.month
    df['DAY'] = df['DATETIME'].dt.day
    df['HOUR'] = df['DATETIME'].dt.hour
    df['MINUTE'] = df['DATETIME'].dt.minute

merge_keys = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']
df_qld_merged = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how='inner'), dataframes_qld)

print(df_qld_merged.head())

   PREDISPATCHSEQNO REGIONID_x  PERIODID  FORECASTDEMAND          LASTCHANGED  \
0        2016123018       QLD1        71         6425.36  2016-12-30 12:31:25   
1        2016123018       QLD1        72         6193.34  2016-12-30 12:31:25   
2        2016123018       QLD1        73         6016.45  2016-12-30 12:31:25   
3        2016123018       QLD1        74         5859.23  2016-12-30 12:31:25   
4        2016123018       QLD1        75         5767.15  2016-12-30 12:31:25   

           DATETIME_x  YEAR  MONTH  DAY  HOUR  MINUTE          DATETIME_y  \
0 2017-01-01 00:00:00  2017      1    1     0       0 2017-01-01 00:00:00   
1 2017-01-01 00:30:00  2017      1    1     0      30 2017-01-01 00:30:00   
2 2017-01-01 01:00:00  2017      1    1     1       0 2017-01-01 01:00:00   
3 2017-01-01 01:30:00  2017      1    1     1      30 2017-01-01 01:30:00   
4 2017-01-01 02:00:00  2017      1    1     2       0 2017-01-01 02:00:00   

   TOTALDEMAND REGIONID_y                      LOC

In [15]:
df_qld_merged.columns

Index(['PREDISPATCHSEQNO', 'REGIONID_x', 'PERIODID', 'FORECASTDEMAND',
       'LASTCHANGED', 'DATETIME_x', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE',
       'DATETIME_y', 'TOTALDEMAND', 'REGIONID_y', 'LOCATION', 'DATETIME',
       'TEMPERATURE'],
      dtype='object')

In [16]:
df_qld_merged.drop(columns=['REGIONID_x', 'DATETIME_x'], inplace=True)
print(df_qld_merged.columns)


Index(['PREDISPATCHSEQNO', 'PERIODID', 'FORECASTDEMAND', 'LASTCHANGED', 'YEAR',
       'MONTH', 'DAY', 'HOUR', 'MINUTE', 'DATETIME_y', 'TOTALDEMAND',
       'REGIONID_y', 'LOCATION', 'DATETIME', 'TEMPERATURE'],
      dtype='object')


In [17]:
df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,TOTALDEMAND,REGIONID_y,LOCATION,DATETIME,TEMPERATURE
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,6728.99,QLD1,Brisbane Archerfield Airport,2017-01-01 00:00:00,26.5
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,6462.14,QLD1,Brisbane Archerfield Airport,2017-01-01 00:30:00,26.3
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,6352.82,QLD1,Brisbane Archerfield Airport,2017-01-01 01:00:00,26.0
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,6261.02,QLD1,Brisbane Archerfield Airport,2017-01-01 01:30:00,25.2
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,6169.66,QLD1,Brisbane Archerfield Airport,2017-01-01 02:00:00,25.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73736,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,6443.62,QLD1,Brisbane Archerfield Airport,2021-03-17 22:00:00,19.6
73737,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,6264.63,QLD1,Brisbane Archerfield Airport,2021-03-17 22:30:00,19.5
73738,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,6144.16,QLD1,Brisbane Archerfield Airport,2021-03-17 23:00:00,19.4
73739,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,5897.64,QLD1,Brisbane Archerfield Airport,2021-03-17 23:30:00,19.6


In [18]:
df_qld_merged['is_weekend'] = df_qld_merged['DATETIME'].dt.dayofweek >= 5

df_qld_merged['season'] = df_qld_merged['DATETIME'].dt.month % 12 // 3
df_qld_merged['season'] = df_qld_merged['season'].map({0: 'Summer', 1: 'Autumn', 2: 'Winter', 3: 'Spring'}, na_action='ignore')


australian_seasons = {
    12: 'Summer', 1: 'Summer', 2: 'Summer',
    3: 'Autumn', 4: 'Autumn', 5: 'Autumn',
    6: 'Winter', 7: 'Winter', 8: 'Winter',
    9: 'Spring', 10: 'Spring', 11: 'Spring'
    }
df_qld_merged['season'] = df_qld_merged['MONTH'].map(australian_seasons)

In [27]:
#df_qld_merged = df_qld_merged.sort_values('DATETIME')
#df_qld_merged['demand_lag1'] = df_qld_merged['TOTALDEMAND'].shift(24)
#df_qld_merged['rolling_mean_24h'] = df_qld_merged['TOTALDEMAND'].rolling(window=24).mean()

In [19]:
df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,TOTALDEMAND,REGIONID_y,LOCATION,DATETIME,TEMPERATURE,is_weekend,season
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,6728.99,QLD1,Brisbane Archerfield Airport,2017-01-01 00:00:00,26.5,True,Summer
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,6462.14,QLD1,Brisbane Archerfield Airport,2017-01-01 00:30:00,26.3,True,Summer
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,6352.82,QLD1,Brisbane Archerfield Airport,2017-01-01 01:00:00,26.0,True,Summer
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,6261.02,QLD1,Brisbane Archerfield Airport,2017-01-01 01:30:00,25.2,True,Summer
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,6169.66,QLD1,Brisbane Archerfield Airport,2017-01-01 02:00:00,25.5,True,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73736,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,6443.62,QLD1,Brisbane Archerfield Airport,2021-03-17 22:00:00,19.6,False,Autumn
73737,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,6264.63,QLD1,Brisbane Archerfield Airport,2021-03-17 22:30:00,19.5,False,Autumn
73738,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,6144.16,QLD1,Brisbane Archerfield Airport,2021-03-17 23:00:00,19.4,False,Autumn
73739,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,5897.64,QLD1,Brisbane Archerfield Airport,2021-03-17 23:30:00,19.6,False,Autumn


In [20]:
df_qld_merged['Cooling'] = df_qld_merged['TEMPERATURE'].apply(lambda x: max(0, x - 24))

df_qld_merged['Heating'] = df_qld_merged['TEMPERATURE'].apply(lambda x: max(0, 20 - x))

print(df_qld_merged[['Cooling', 'Heating']].head())  # Displaying the first few rows to verify


   Cooling  Heating
0      2.5      0.0
1      2.3      0.0
2      2.0      0.0
3      1.2      0.0
4      1.5      0.0


In [21]:
df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,TOTALDEMAND,REGIONID_y,LOCATION,DATETIME,TEMPERATURE,is_weekend,season,Cooling,Heating
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,6728.99,QLD1,Brisbane Archerfield Airport,2017-01-01 00:00:00,26.5,True,Summer,2.5,0.0
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,6462.14,QLD1,Brisbane Archerfield Airport,2017-01-01 00:30:00,26.3,True,Summer,2.3,0.0
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,6352.82,QLD1,Brisbane Archerfield Airport,2017-01-01 01:00:00,26.0,True,Summer,2.0,0.0
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,6261.02,QLD1,Brisbane Archerfield Airport,2017-01-01 01:30:00,25.2,True,Summer,1.2,0.0
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,6169.66,QLD1,Brisbane Archerfield Airport,2017-01-01 02:00:00,25.5,True,Summer,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73736,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,6443.62,QLD1,Brisbane Archerfield Airport,2021-03-17 22:00:00,19.6,False,Autumn,0.0,0.4
73737,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,6264.63,QLD1,Brisbane Archerfield Airport,2021-03-17 22:30:00,19.5,False,Autumn,0.0,0.5
73738,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,6144.16,QLD1,Brisbane Archerfield Airport,2021-03-17 23:00:00,19.4,False,Autumn,0.0,0.6
73739,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,5897.64,QLD1,Brisbane Archerfield Airport,2021-03-17 23:30:00,19.6,False,Autumn,0.0,0.4


In [37]:
df_qld_merged.columns

Index(['PREDISPATCHSEQNO', 'PERIODID', 'FORECASTDEMAND', 'LASTCHANGED', 'YEAR',
       'MONTH', 'DAY', 'HOUR', 'MINUTE', 'DATETIME_y', 'TOTALDEMAND',
       'REGIONID_y', 'LOCATION', 'DATETIME', 'TEMPERATURE', 'is_weekend',
       'season', 'Cooling', 'Heating'],
      dtype='object')

In [22]:
numerical_features = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'TEMPERATURE', 'Heating', 'Cooling']
categorical_features = ['season']
binary_features = ['is_weekend']  

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('bin', 'passthrough', binary_features)
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

X = df_qld_merged.drop(['TOTALDEMAND'], axis=1)
y = df_qld_merged['TOTALDEMAND']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')



Mean Squared Error: 336813.52825416595
R^2 Score: 0.5788485826472984


In [24]:
pipeline.fit(X_train, y_train)

# Using get_feature_names_out for compatibility with newer scikit-learn versions
ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(input_features=categorical_features)
feature_names_transformed = numerical_features + list(ohe_feature_names) + binary_features

coefficients = pipeline.named_steps['regressor'].coef_

feature_importance = pd.Series(coefficients, index=feature_names_transformed)

feature_importance_sorted = feature_importance.abs().sort_values(ascending=False)

print("Feature Importance Ranked:")
print(feature_importance_sorted)


Feature Importance Ranked:
season_Summer    1.016931e+14
season_Autumn    1.016931e+14
season_Spring    1.016931e+14
season_Winter    1.016931e+14
HOUR             5.017408e+02
is_weekend       4.094662e+02
TEMPERATURE      1.680186e+02
Heating          1.573034e+02
Cooling          1.340670e+02
MONTH            9.036869e+01
YEAR             4.930848e+01
MINUTE           3.074791e+00
DAY              2.988440e+00
dtype: float64


In [25]:
entire_data_predictions = pipeline.predict(X)

df_qld_merged['PREDICTED_TOTALDEMAND'] = entire_data_predictions

In [26]:
df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,TOTALDEMAND,REGIONID_y,LOCATION,DATETIME,TEMPERATURE,is_weekend,season,Cooling,Heating,PREDICTED_TOTALDEMAND
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,6728.99,QLD1,Brisbane Archerfield Airport,2017-01-01 00:00:00,26.5,True,Summer,2.5,0.0,5861.281250
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,6462.14,QLD1,Brisbane Archerfield Airport,2017-01-01 00:30:00,26.3,True,Summer,2.3,0.0,5848.109375
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,6352.82,QLD1,Brisbane Archerfield Airport,2017-01-01 01:00:00,26.0,True,Summer,2.0,0.0,5885.390625
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,6261.02,QLD1,Brisbane Archerfield Airport,2017-01-01 01:30:00,25.2,True,Summer,1.2,0.0,5814.250000
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,6169.66,QLD1,Brisbane Archerfield Airport,2017-01-01 02:00:00,25.5,True,Summer,1.5,0.0,5909.515625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73736,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,6443.62,QLD1,Brisbane Archerfield Airport,2021-03-17 22:00:00,19.6,False,Autumn,0.0,0.4,6825.984375
73737,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,6264.63,QLD1,Brisbane Archerfield Airport,2021-03-17 22:30:00,19.5,False,Autumn,0.0,0.5,6834.031250
73738,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,6144.16,QLD1,Brisbane Archerfield Airport,2021-03-17 23:00:00,19.4,False,Autumn,0.0,0.6,6902.187500
73739,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,5897.64,QLD1,Brisbane Archerfield Airport,2021-03-17 23:30:00,19.6,False,Autumn,0.0,0.4,6904.578125


In [27]:
df_qld_merged['Error'] = df_qld_merged['PREDICTED_TOTALDEMAND'] - df_qld_merged['TOTALDEMAND']

df_qld_merged['Error Ratio'] = df_qld_merged['Error'] / df_qld_merged['PREDICTED_TOTALDEMAND']

df_qld_merged


,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,...,LOCATION,DATETIME,TEMPERATURE,is_weekend,season,Cooling,Heating,PREDICTED_TOTALDEMAND,Error,Error Ratio
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,...,Brisbane Archerfield Airport,2017-01-01 00:00:00,26.5,True,Summer,2.5,0.0,5861.281250,-867.708750,-0.148041
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,...,Brisbane Archerfield Airport,2017-01-01 00:30:00,26.3,True,Summer,2.3,0.0,5848.109375,-614.030625,-0.104996
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,...,Brisbane Archerfield Airport,2017-01-01 01:00:00,26.0,True,Summer,2.0,0.0,5885.390625,-467.429375,-0.079422
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,...,Brisbane Archerfield Airport,2017-01-01 01:30:00,25.2,True,Summer,1.2,0.0,5814.250000,-446.770000,-0.076841
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,...,Brisbane Archerfield Airport,2017-01-01 02:00:00,25.5,True,Summer,1.5,0.0,5909.515625,-260.144375,-0.044021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73736,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,...,Brisbane Archerfield Airport,2021-03-17 22:00:00,19.6,False,Autumn,0.0,0.4,6825.984375,382.364375,0.056016
73737,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,...,Brisbane Archerfield Airport,2021-03-17 22:30:00,19.5,False,Autumn,0.0,0.5,6834.031250,569.401250,0.083319
73738,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,...,Brisbane Archerfield Airport,2021-03-17 23:00:00,19.4,False,Autumn,0.0,0.6,6902.187500,758.027500,0.109824
73739,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,...,Brisbane Archerfield Airport,2021-03-17 23:30:00,19.6,False,Autumn,0.0,0.4,6904.578125,1006.938125,0.145836


In [28]:
average_error = df_qld_merged['Error'].mean()
average_error_ratio = df_qld_merged['Error Ratio'].mean()
std_error = df_qld_merged['Error'].std()
std_error_ratio = df_qld_merged['Error Ratio'].std()

print(f'Average Error: {average_error}')
print(f'Average Error Ratio: {average_error_ratio}')
print(f'Standard Deviation of Error: {std_error}')
print(f'Standard Deviation of Error Ratio: {std_error_ratio}')



Average Error: 1.2159387671037811
Average Error Ratio: -2.226241127342202e-05
Standard Deviation of Error: 579.0694124592313
Standard Deviation of Error Ratio: 0.091543564649393


In [29]:
df_qld_merged[['Error Ratio','Error']].describe()

,Error Ratio,Error
count,73741.000000,73741.000000
mean,-0.000022,1.215939
std,0.091544,579.069412
min,-0.377347,-2457.081250
25%,-0.063016,-387.112500
50%,0.003698,22.638125
75%,0.063487,389.393125
max,0.332004,2094.930625


In [30]:
df_qld_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73741 entries, 0 to 73740
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PREDISPATCHSEQNO       73741 non-null  int64         
 1   PERIODID               73741 non-null  int64         
 2   FORECASTDEMAND         73741 non-null  float64       
 3   LASTCHANGED            73741 non-null  object        
 4   YEAR                   73741 non-null  int32         
 5   MONTH                  73741 non-null  int32         
 6   DAY                    73741 non-null  int32         
 7   HOUR                   73741 non-null  int32         
 8   MINUTE                 73741 non-null  int32         
 9   DATETIME_y             73741 non-null  datetime64[ns]
 10  TOTALDEMAND            73741 non-null  float64       
 11  REGIONID_y             73741 non-null  object        
 12  LOCATION               73741 non-null  object        
 13  D

## Creating a NN based onthe intra day data

In [31]:
from sklearn.neural_network import MLPRegressor

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000, random_state=42))
])

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


Mean Squared Error: 60078.817086084164
R^2 Score: 0.9248774860682423


In [32]:
df_qld_merged['PREDICTED_TOTALDEMAND_NN'] = pipeline.predict(X)

df_qld_merged['Error_NN'] = df_qld_merged['PREDICTED_TOTALDEMAND_NN'] - df_qld_merged['TOTALDEMAND']

df_qld_merged['Error Ratio_NN'] = df_qld_merged['Error_NN'] / df_qld_merged['PREDICTED_TOTALDEMAND_NN']

df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,...,is_weekend,season,Cooling,Heating,PREDICTED_TOTALDEMAND,Error,Error Ratio,PREDICTED_TOTALDEMAND_NN,Error_NN,Error Ratio_NN
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,...,True,Summer,2.5,0.0,5861.281250,-867.708750,-0.148041,6428.503263,-300.486737,-0.046743
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,...,True,Summer,2.3,0.0,5848.109375,-614.030625,-0.104996,6278.686077,-183.453923,-0.029219
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,...,True,Summer,2.0,0.0,5885.390625,-467.429375,-0.079422,6139.132928,-213.687072,-0.034807
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,...,True,Summer,1.2,0.0,5814.250000,-446.770000,-0.076841,5961.921178,-299.098822,-0.050168
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,...,True,Summer,1.5,0.0,5909.515625,-260.144375,-0.044021,5866.426129,-303.233871,-0.051690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73736,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,...,False,Autumn,0.0,0.4,6825.984375,382.364375,0.056016,6467.588065,23.968065,0.003706
73737,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,...,False,Autumn,0.0,0.5,6834.031250,569.401250,0.083319,6319.661610,55.031610,0.008708
73738,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,...,False,Autumn,0.0,0.6,6902.187500,758.027500,0.109824,6162.518542,18.358542,0.002979
73739,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,...,False,Autumn,0.0,0.4,6904.578125,1006.938125,0.145836,6045.667681,148.027681,0.024485


In [33]:
df_qld_merged['Error_Prof'] = df_qld_merged['FORECASTDEMAND'] - df_qld_merged['TOTALDEMAND']

df_qld_merged['Error Ratio_Prof'] = df_qld_merged['Error_Prof'] / df_qld_merged['FORECASTDEMAND']

df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,...,Cooling,Heating,PREDICTED_TOTALDEMAND,Error,Error Ratio,PREDICTED_TOTALDEMAND_NN,Error_NN,Error Ratio_NN,Error_Prof,Error Ratio_Prof
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,...,2.5,0.0,5861.281250,-867.708750,-0.148041,6428.503263,-300.486737,-0.046743,-303.63,-0.047255
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,...,2.3,0.0,5848.109375,-614.030625,-0.104996,6278.686077,-183.453923,-0.029219,-268.80,-0.043401
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,...,2.0,0.0,5885.390625,-467.429375,-0.079422,6139.132928,-213.687072,-0.034807,-336.37,-0.055908
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,...,1.2,0.0,5814.250000,-446.770000,-0.076841,5961.921178,-299.098822,-0.050168,-401.79,-0.068574
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,...,1.5,0.0,5909.515625,-260.144375,-0.044021,5866.426129,-303.233871,-0.051690,-402.51,-0.069794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73736,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,...,0.0,0.4,6825.984375,382.364375,0.056016,6467.588065,23.968065,0.003706,219.02,0.032873
73737,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,...,0.0,0.5,6834.031250,569.401250,0.083319,6319.661610,55.031610,0.008708,253.19,0.038846
73738,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,...,0.0,0.6,6902.187500,758.027500,0.109824,6162.518542,18.358542,0.002979,222.91,0.035010
73739,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,...,0.0,0.4,6904.578125,1006.938125,0.145836,6045.667681,148.027681,0.024485,267.41,0.043375


In [34]:
df_qld_merged[['Error Ratio_NN','Error_NN','Error Ratio','Error', 'Error_Prof','Error Ratio_Prof']].describe()

,Error Ratio_NN,Error_NN,Error Ratio,Error,Error_Prof,Error Ratio_Prof
count,73741.000000,73741.000000,73741.000000,73741.000000,73741.000000,73741.000000
mean,0.003158,16.835563,-0.000022,1.215939,-9.195399,-0.001591
std,0.037990,240.821608,0.091544,579.069412,158.560838,0.025200
min,-0.264398,-1708.926786,-0.377347,-2457.081250,-918.000000,-0.187521
25%,-0.016202,-101.085133,-0.063016,-387.112500,-92.740000,-0.015082
50%,0.003703,22.499228,0.003698,22.638125,-3.060000,-0.000509
75%,0.022996,138.960491,0.063487,389.393125,81.200000,0.013414
max,0.202655,1506.615957,0.332004,2094.930625,1170.470000,0.149933


# Creating a stacked model using GBM, Lin Reg and MLP to see if we can get better performance than the NN Alone

In [35]:
mape = (df_qld_merged['Error_NN'].abs() / df_qld_merged['TOTALDEMAND'].abs()).mean() * 100

print(f'MAPE: {mape}%')

MAPE: 2.7534752459101237%


In [39]:
numerical_features = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'TEMPERATURE', 'Heating', 'Cooling']
categorical_features = ['season']
binary_features = ['is_weekend']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('bin', 'passthrough', binary_features)
    ])

X = df_qld_merged.drop(['TOTALDEMAND'], axis=1)
y = df_qld_merged['TOTALDEMAND']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


base_models = [
    ('lr', Pipeline([('preprocessor', clone(preprocessor)),
                     ('regressor', LinearRegression())])),
    ('gbm', Pipeline([('preprocessor', clone(preprocessor)),
                      ('regressor', GradientBoostingRegressor(random_state=42))])),
    ('mlp', Pipeline([('preprocessor', clone(preprocessor)),
                      ('regressor', MLPRegressor(hidden_layer_sizes=(64, 62), 
                                                 activation='relu', 
                                                 solver='adam', 
                                                 max_iter=1000, 
                                                 random_state=42))]))
]

stacked_model = StackingRegressor(
    estimators=base_models,
    final_estimator=LinearRegression()
)

stacked_model.fit(X_train, y_train)

y_pred = stacked_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


Mean Squared Error: 55467.67189338839
R^2 Score: 0.9306432590275086


In [41]:
X = df_qld_merged.drop(['TOTALDEMAND'], axis=1, errors='ignore')

df_qld_merged['Predicted_Demand_Stacked_Model'] = stacked_model.predict(X)

df_qld_merged['Error_Rate'] = df_qld_merged['Predicted_Demand_Stacked_Model'] - df_qld_merged['TOTALDEMAND']

df_qld_merged['Error_Ratio'] = df_qld_merged['Error_Rate'] / df_qld_merged['TOTALDEMAND']

print(df_qld_merged[['TOTALDEMAND', 'Predicted_Demand_Stacked_Model', 'Error_Rate', 'Error_Ratio']].head())


   TOTALDEMAND  Predicted_Demand_Stacked_Model  Error_Rate  Error_Ratio
0      6728.99                     6617.777968 -111.212032    -0.016527
1      6462.14                     6399.017543  -63.122457    -0.009768
2      6352.82                     6299.879233  -52.940767    -0.008333
3      6261.02                     6008.870866 -252.149134    -0.040273
4      6169.66                     6013.803457 -155.856543    -0.025262


In [42]:
df_qld_merged[['Error Ratio_NN','Error_NN','Error Ratio','Error', 'Error_Prof','Error Ratio_Prof','Error_Rate','Error_Ratio']].describe()

,Error Ratio_NN,Error_NN,Error Ratio,Error,Error_Prof,Error Ratio_Prof,Error_Rate,Error_Ratio
count,73741.000000,73741.000000,73741.000000,73741.000000,73741.000000,73741.000000,73741.000000,73741.000000
mean,0.003158,16.835563,-0.000022,1.215939,-9.195399,-0.001591,-3.837617,0.000713
std,0.037990,240.821608,0.091544,579.069412,158.560838,0.025200,229.892735,0.036411
min,-0.264398,-1708.926786,-0.377347,-2457.081250,-918.000000,-0.187521,-1811.289289,-0.225231
25%,-0.016202,-101.085133,-0.063016,-387.112500,-92.740000,-0.015082,-112.347313,-0.018000
50%,0.003703,22.499228,0.003698,22.638125,-3.060000,-0.000509,-0.811666,-0.000130
75%,0.022996,138.960491,0.063487,389.393125,81.200000,0.013414,110.686337,0.018460
max,0.202655,1506.615957,0.332004,2094.930625,1170.470000,0.149933,1455.321742,0.250121
